In [1]:
from typing import List, Dict, Literal, Union
import datetime, time 
import numpy as np 
import pandas as pd
import RPi.GPIO as GPIO
from multiprocessing import Pipe, Process
from multiprocessing.connection import Connection

from data_collection import Logger, LoggerSet

from components.two_wheels import TwoWheels, speed_proportion_control
from components.ir_sensor import IRreciever, code2command

from components.camera import Camera

/tmp/ipykernel_4722/582279248.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
code2command = {
    16769055: 'V-',   
    16754775: 'V+',
    16720605: 'L+',   # R Wheel speed up = steer left
    16712445: 'R+',  
    16761405: 'LR',
    16748655: 'stop',
}

def get_ir_2wheels_adaptor():
    data = {}
    data['speed'] = 0
    data['proportion'] = 0.5 # left = proportion * speed

    def step(command):
        
        if command == 'stop':
            data['speed'] = 0
            
        elif command == 'L+':
            data['proportion'] -= 0.05
        
        elif command == 'R+':
            data['proportion'] += 0.05        
        
        elif command == 'V+':
            data['speed'] += 10    
            
        elif command == 'V-':
            data['speed'] -= 5       
        elif command == 'LR':
            data['proportion'] = 0.5       
        else: 
            pass

        left, right, warnings = speed_proportion_control(data['proportion'], data['speed'])
        return left, right
    return step

In [3]:
logger_set = LoggerSet('../log/temp', True)

ir_p, ir_out = IRreciever.start(8, code2command, logger_set, name='IR', save_interval=30)
wheels_p, wheels_in = TwoWheels.start(33, 32, logger_set, name='Wheels', save_interval=30)
adaptor = get_ir_2wheels_adaptor()
cam_p = Camera.start(0, logger_set, name='camera', save_interval=30)

/home/kawa/projects/car1/src/components/two_wheels.py:12: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(pin, GPIO.OUT)
Process Process-2:
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/kawa/projects/car1/src/components/two_wheels.py", line 64, in main
    time.sleep(0.1)
  File "/usr/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
  File "/home/kawa/projects/car1/src/components/ir_sensor.py", line 81, in main
    command = component.poll_as_command()
  File "/home/kawa/p

In [4]:
def main():

    while True: 
        command = ir_out.recv()
        left, right = adaptor(command)
        
        wheels_in.send((left, right))
            
main()
    

KeyboardInterrupt: 